# Zuhlke - NewAutoMotive Hackathon



## Data sources

### Setup

In [5]:
import requests
import pandas as pd
from pandas_ods_reader import read_ods
from pandas_profiling import ProfileReport

### 1. New Vehicle Registrations by vehicle type, quarterly

The Department for Transport (DfT) and their statistical team publishes a large amount of vehicle information.  
The main page for this information is here: https://www.gov.uk/government/collections/vehicles-statistics  
We've selected a few tables that are of particular interest. The code below downloads the data file to the `data` folder, and pulls a relevant part of the spreadsheet into a dataframe.

In [43]:
file_url =  'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/917425/veh0253.ods'
filepath = f'data/veh0253.ods'

r = requests.get(file_url)
with open(filepath, 'wb') as f:
    f.write(r.content)

In [44]:
df_raw = read_ods(filepath, 1)

headers = df_raw.iloc[6, :].values
cols = dict(zip(df_raw.columns, df_raw.iloc[6, :].values))

df1 = (df_raw
          .iloc[27: 105, :]
          .rename(columns=cols)
          .reset_index(drop=True)
         )

In [45]:
df1.columns = ['Date', 'Petrol', 'Diesel', 'Hybrid Electric',
       'Plug-in Hybrid Electric', 'Battery Electric',
       'Range-Extended Electric', 'Fuel Cell Electric', 'Gas', 'Other',
       'Total', 'Alternative Fuels']

In [46]:
df1.head(5)

,Date,Petrol,Diesel,Hybrid Electric,Plug-in Hybrid Electric,Battery Electric,Range-Extended Electric,Fuel Cell Electric,Gas,Other,Total,Alternative Fuels
0,2001 Q1,598.295,108.329,0.248,0,0.015,0,0,1.035,0.001,707.923,1.299
1,2001 Q2,512.979,105.089,0.157,0,0.009,0,0,0.321,0.004,618.559,0.491
2,2001 Q3,596.655,132.522,0.155,0,0.043,0,0,0.18,0.002,729.557,0.38
3,2001 Q4,414.583,114.706,0.075,0,0.01,0,0,0.567,0.002,529.943,0.654
4,2002 Q1,591.202,166.614,0.074,0,0.004,0,0,0.794,0,758.688,0.872


In [47]:
df1.tail(5)

,Date,Petrol,Diesel,Hybrid Electric,Plug-in Hybrid Electric,Battery Electric,Range-Extended Electric,Fuel Cell Electric,Gas,Other,Total,Alternative Fuels
73,2019 Q2,370.189,154.288,25.047,6.548,6.238,0.013,0.012,0.009,0.001,562.345,37.868
74,2019 Q3,387.98,147.991,32.441,7.845,12.727,0.026,0.028,0.012,0.002,589.052,53.081
75,2019 Q4,285.114,113.55,26.604,11.934,12.68,0.007,0.014,0.002,0.001,449.906,51.242
76,2020 Q1,306.607,103.165,38.253,13.518,18.086,0.01,0.011,0.047,0,479.697,69.925
77,2020 Q2,105.211,29.892,14.498,5.736,12.639,0.006,0.007,0.074,0,168.063,32.96


In [48]:
# what is the % share of battery electric vehicles in 2020 Q2?
round((df1.tail(1)[['Battery Electric']].sum().sum() / df1[df1['Date']=='2020 Q2']['Total']).values[0] * 100, 2)

7.52

In [50]:
profile1 = ProfileReport(df1, title='Pandas Profiling Report') # , explorative=True
profile1

In [53]:
profile1.to_file("1_New Vehicle Registrations by vehicle type, quarterly.html")

## 2. Miles travelled by vehicle make, model and fuelType 2015-2019

In [21]:
if False:
    file_url =  'https://storage.googleapis.com/new_automotive/yearly_mileage_make-model-fuelType_2015-2019.csv'
    filepath = f'data/yearly_mileage_make-model-fuelType_2015-2019.csv'

    r = requests.get(file_url)
    with open(filepath, 'wb') as f:
        f.write(r.content)

In [22]:
df2 = pd.read_csv('data/yearly_mileage_make-model-fuelType_2015-2019.csv')

In [23]:
df2.head()

,Unnamed: 0,mileage,y,make,model,fuelType
0,0,1.073740e+10,2017,FORD,TRANSIT,Diesel
1,1,1.049419e+10,2018,FORD,TRANSIT,Diesel
2,2,1.016843e+10,2016,FORD,TRANSIT,Diesel
3,3,9.952579e+09,2019,FORD,TRANSIT,Diesel
4,4,9.661517e+09,2018,FORD,FIESTA,Petrol


In [24]:
df2.describe()

,Unnamed: 0,mileage,y
count,598811.000000,5.988110e+05,598811.000000
mean,299405.000000,2.767434e+06,2016.990803
std,172861.990359,7.540581e+07,1.414307
min,0.000000,-1.164153e-10,2015.000000
25%,149702.500000,1.307663e+03,2016.000000
50%,299405.000000,6.771513e+03,2017.000000
75%,449107.500000,3.012562e+04,2018.000000
max,598810.000000,1.073740e+10,2019.000000


In [32]:
# Takes very long-- never completed
#profile2 = ProfileReport(df2, title='Pandas Profiling Report') # , explorative=True
#profile2

## 3. CO2 Emissions over time

Two datasets, one with just age and fueltype, and one with make and model.  
These have been aggregated from a set of 1.5m vehicles sampled from the DVLA vehicle checker API.  
Try it yourself here: https://vehicleenquiry.service.gov.uk/

**Data quality problems**: CO2 values: **30 %** missing, **30 %** 0.0 (mostly PETROL and DIESEL cars)

In [10]:
if False:
    
    file_url =  'https://storage.googleapis.com/new_automotive/avg_co2Emissions_by_fuelType_yearOfManufacture.csv'
    filepath = f'data/avg_co2Emissions_by_fuelType_yearOfManufacture.csv'

    r = requests.get(file_url)
    with open(filepath, 'wb') as f:
        f.write(r.content)

In [11]:
file_url =  'https://storage.googleapis.com/new_automotive/avg_co2Emissions_by_fuelType_yearOfManufacture_make_model.csv'
filepath = f'data/avg_co2Emissions_by_fuelType_yearOfManufacture_make_model.csv'

r = requests.get(file_url)
with open(filepath, 'wb') as f:
    f.write(r.content)

In [12]:
df3 = pd.read_csv(filepath)

In [13]:
df3.head()

,avg_co2Emissions,avg_engineCapacity,fuelType,yearOfManufacture,make,model
0,NaN,3995.250000,PETROL,1967.0,ASTON MARTIN,DB6
1,NaN,6750.000000,PETROL,1976.0,ROLLS ROYCE,SILVER SHADOW 1
2,NaN,4235.000000,PETROL,1974.0,DAIMLER,DS420
3,NaN,2303.222222,PETROL,1987.0,BMW,3 SERIES
4,NaN,1414.100000,PETROL,1973.0,VOLKSWAGEN,BEETLE


In [22]:
df3_no_co2=df3.loc[df3['avg_co2Emissions'].isnull()]
df3_no_co2

,avg_co2Emissions,avg_engineCapacity,fuelType,yearOfManufacture,make,model
0,NaN,3995.250000,PETROL,1967.0,ASTON MARTIN,DB6
1,NaN,6750.000000,PETROL,1976.0,ROLLS ROYCE,SILVER SHADOW 1
2,NaN,4235.000000,PETROL,1974.0,DAIMLER,DS420
3,NaN,2303.222222,PETROL,1987.0,BMW,3 SERIES
4,NaN,1414.100000,PETROL,1973.0,VOLKSWAGEN,BEETLE
...,...,...,...,...,...,...
47562,NaN,1998.000000,PETROL,2015.0,GBS ZERO RACE,NaN
47661,NaN,1998.000000,PETROL,2015.0,YATES,7
48042,NaN,1800.000000,PETROL,2016.0,HAYNES ROADSTER,NaN
48936,NaN,3456.000000,PETROL,2017.0,LOTUS,UNKNOWN


In [26]:
df3_no_co2[df3_no_co2.fuelType != 'PETROL']

,avg_co2Emissions,avg_engineCapacity,fuelType,yearOfManufacture,make,model
13,NaN,2494.666667,DIESEL,1988.0,LAND ROVER,DEFENDER
51,NaN,2400.000000,DIESEL,1988.0,VOLKSWAGEN,LT
72,NaN,2286.000000,DIESEL,1976.0,LAND ROVER,88 - 4 CYL
76,NaN,2588.000000,DIESEL,1971.0,LAND ROVER,RANGE ROVER
88,NaN,2500.000000,DIESEL,NaN,LAND ROVER,SERIES 3
...,...,...,...,...,...,...
42685,NaN,904.000000,DIESEL,2013.0,POLARIS,UNKNOWN
45305,NaN,1996.000000,HYBRID ELECTRIC,2014.0,LEXUS,CT200H
45459,NaN,2360.000000,HYBRID ELECTRIC,2014.0,TOYOTA,ESTIMA
46154,NaN,2495.000000,DIESEL,2014.0,FOERS IBEX F8,NaN


In [27]:
df3_co2_0=df3.loc[df3['avg_co2Emissions']==0]
df3_co2_0

,avg_co2Emissions,avg_engineCapacity,fuelType,yearOfManufacture,make,model
11,0.0,1292.250000,PETROL,NaN,VOLKSWAGEN,NaN
14,0.0,NaN,PETROL,1966.0,VOLKSWAGEN,MICRO BUS
25,0.0,182.000000,PETROL,1969.0,LAMBRETTA,GP150
31,0.0,398.000000,PETROL,1977.0,YAMAHA,DT400MX
42,0.0,149.333333,PETROL,1964.0,LAMBRETTA,LI150
...,...,...,...,...,...,...
49228,0.0,223.000000,PETROL,2017.0,AJS,XGJ
49234,0.0,1262.000000,PETROL,2017.0,DUCATI,X DIAVEL
49235,0.0,124.000000,PETROL,2017.0,PIAGGIO,LIBERTY 125
49236,0.0,6700.000000,DIESEL,2017.0,DODGE,UNKNOWN


In [31]:
df3_co2_0.groupby(df3_co2_0.fuelType).count()

,avg_co2Emissions,avg_engineCapacity,yearOfManufacture,make,model
fuelType,,,,,
DIESEL,2938,2937,2934,2938,2751
ELECTRICITY,128,103,128,128,124
GAS,11,11,11,11,11
GAS BI-FUEL,126,126,126,126,117
HYBRID ELECTRIC,26,26,26,26,26
OTHER,3,3,3,3,3
PETROL,11859,11858,11788,11859,11409
PETROL/GAS,10,10,10,10,5


In [14]:
profile3 = ProfileReport(df3, title='Pandas Profiling Report') # , explorative=True
profile3

In [52]:
profile3.to_file("3_co2Emissions.html")

## 4. EV Growth by Local Authority (LA) over time

In [36]:
file_url = 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/853462/veh0131.ods'
filepath = f'data/veh0131.ods'

r = requests.get(file_url)
with open(filepath, 'wb') as f:
    f.write(r.content)

In [37]:
df_raw = read_ods(filepath, 1)

headers = df_raw.iloc[5, :].values
cols = dict(zip(df_raw.columns, df_raw.iloc[5, :].values))

df4 = (df_raw
      .iloc[6:478, :]
      .rename(columns=cols)
      .reset_index(drop=True)
     )

# dropping null regions
df4 = df4[df4[['ONS LA Code', 'Region/Local Authority']].isnull().sum(axis=1) == 0]

df4.head()

,ONS LA Code,Region/Local Authority,2020 Q2,2020 Q1,2019 Q4,2019 Q3,2019 Q2,2019 Q1,2018 Q4,2018 Q3,...,2014 Q1,2013 Q4,2013 Q3,2013 Q2,2013 Q1,2012 Q4,2012 Q3,2012 Q2,2012 Q1,2011 Q4
0,K02000001,United Kingdom,300931,283910,253957,230811,211440,199886,186407,172220,...,13616,11868,10905,10122,9213,8606,7843,7211,6563,6228
1,K03000001,Great Britain,297305,280453,250865,227881,208677,197217,183909,169827,...,13427,11706,10760,9995,9119,8530,7778,7169,6537,6206
2,E92000001,England,268326,255106,225804,204890,187854,177784,166265,153128,...,11933,10426,9578,8849,8135,7544,6998,6420,5909,5705
3,E12000001,North East,4666,4565,4112,3917,3613,3448,3384,3198,...,413,343,321,290,256,235,226,202,182,189
4,E06000047,County Durham UA,930,906,786,745,714,673,671,631,...,69,58,53,39,30,27,26,22,20,39


In [40]:
#ValueError: shape mismatch: value array of shape (33,) could not be broadcast to indexing result of shape (32,)
#profile4 = ProfileReport(df4, title='Pandas Profiling Report') # , explorative=True
#profile4

This data blends Local Authority and Region codes.  
To disentangle, the codes need to map to either a LA or a Region.  

LA codes: https://data.gov.uk/dataset/24d87ad2-0fa9-4b35-816a-89f9d92b0042/local-authority-districts-april-2020-names-and-codes-in-the-united-kingdom

GeoJson and CSV is available.


## 5. What type of vehicles are there?

A DfT spreadsheet provides vehicle Make and Model by vehicle type (the tabs are different vehicle types). But some Makes / Models appear across multiple categories. Can we create a matching for Make / Model to a predominant vehicle type (maybe by count), and use that to slice other datasets by vehicle type?  

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/917200/veh0120.ods





We also have an aggregated set of 35m active vehicles (vehicles with MOTs in date at around September 2020).



In [6]:
file_url = 'https://storage.googleapis.com/new_automotive/active_vehicle_counts.csv'
filepath = f'data/active_vehicle_counts.csv'

r = requests.get(file_url)
with open(filepath, 'wb') as f:
    f.write(r.content)

In [7]:
df5 = pd.read_csv(filepath)

In [8]:
df5.head()

,count,avg_engineSize,make,model,avg_year,fuelType,primaryColour
0,34243,NaN,NaN,NaN,NaN,NaN,NaN
1,2949,996.03,SUZUKI,ALTO,2011.40,Petrol,Pink
2,33,399.85,BSA,UNKNOWN,1955.94,Petrol,Not Stated
3,1,2500.00,LDV,CONVOY,2001.00,Diesel,Pink
4,51,996.00,SUZUKI,ALTO SZ4 AUTO,2012.39,Petrol,Pink


In [9]:
profile5 = ProfileReport(df5, title='Pandas Profiling Report') # , explorative=True
profile5

In [51]:
profile5.to_file("5_active_vehicle_counts.html")